# Installation Notebook for Ubuntu Desktop 22.04

# Installation:

```shell
sudo apt-get install -y git python3.10-venv

cd ~/git/repos

git clone https://github.com/michimussato/OpenStudioLandscapes-Installer.git

cd OpenStudioLandscapes-Installer/ubuntu/22.04

python3.10 -m venv .venv
source .venv/bin/activate

pip install --upgrade pip setuptools setuptools_scm wheel

pip install notebook
```

```shell
jupyter notebook
```

## Step 1

Constants

In [1]:
SHELL_SCRIPTS_PREFIX: str = "ubuntu_desktop_2204__"
URL_HARBOR: str = "http://harbor.farm.evil:80"
ADMIN_HARBOR: str = "admin"
PASSWORD_HARBOR: str = "Harbor12345"

## Step 2

Set `OPENSTUDIOLANDSCAPES` variable so that the installer knows where to pull the source to.

In [2]:
import pathlib
from getpass import getuser


OPENSTUDIOLANDSCAPES_DIR = None

while OPENSTUDIOLANDSCAPES_DIR is None:
    default_openstudiolandscapes_base = "~/git/repos"
    default_openstudiolandscapes_subdir = "OpenStudioLandscapes"

    openstudiolandscapes_base = pathlib.Path(input(f"Install base dir ({default_openstudiolandscapes_base}): ".strip()) or default_openstudiolandscapes_base).expanduser()

    if not openstudiolandscapes_base.is_absolute():
        print(f"ERROR: Directory {openstudiolandscapes_base.as_posix()} is not absolute (~ is allowed).")
        continue
    if not openstudiolandscapes_base.exists():
        print(f"ERROR: Directory {openstudiolandscapes_base.as_posix()} does not exist. Create it first.")
        continue
    if openstudiolandscapes_base.is_file():
        print(f"ERROR: Install Directory {openstudiolandscapes_base.as_posix()} is a file. Cannot continue.")
        continue

    try:
        probe = pathlib.Path(openstudiolandscapes_base / ".openstudiolandscapes_probe")
        probe.mkdir(parents=True, exist_ok=True)
        probe.rmdir()
        del probe
    except Exception as e:
        print(f"ERROR: Unable to write to {openstudiolandscapes_base.as_posix()}: {e}")
        print(f"Make sure we have write permissions to `{openstudiolandscapes_base.as_posix()}` so that we can create a subdirectory.")
        print(f"i.e `sudo chown -R {getuser()}: {openstudiolandscapes_base.as_posix()}`.")
        continue

    openstudiolandscapes_subdir = input(f"Install sub dir ({default_openstudiolandscapes_subdir}): ".strip()) or default_openstudiolandscapes_subdir
    # Todo:
    #  - [ ] Maybe some more checks here
    OPENSTUDIOLANDSCAPES_DIR = pathlib.Path(openstudiolandscapes_base, openstudiolandscapes_subdir).expanduser()

Install base dir (~/git/repos): ~/git/repos/TEST
Install sub dir (OpenStudioLandscapes): 


## Step 2

Source basic functions:
- `script_run()`
- `_get_terminal_size()`
- `sudo_pass()`

In [3]:
import shutil
import shlex
import subprocess
from getpass import getpass
from typing import Tuple


def script_run(
    sudo: bool = False,
    *,
    script: pathlib.Path,
) -> Tuple[bytes, bytes]:

    cmd = [
        shutil.which("bash"),
        script.as_posix(),
    ]

    if sudo:
        cmd.insert(0, shutil.which("sudo"))
        cmd.insert(1, "--stdin")
        cmd.insert(2, "--reset-timestamp")

    with open(script.as_posix(), "r") as f:
        lines = f.readlines()
        print(" COMMAND ".center(_get_terminal_size()[0], "-"))
        print(f" {shlex.join(cmd)} ".center(_get_terminal_size()[0], " "))
        print(" SCRIPT START ".center(_get_terminal_size()[0], "-"))
        lno = 0
        len_ = len(str(len(lines)))
        for l in lines:
            lno += 1
            print(f"{str(lno).ljust(len_)}: {l.rstrip()}")
        print(" SCRIPT END ".center(_get_terminal_size()[0], "-"))

    try:
        proc = subprocess.run(
            cmd,
            input=None if not sudo else sudo_pass(),
            check=True,
            # cwd=script_prep.parent.as_posix(),
            # env=os.environ,
        )
    except subprocess.CalledProcessError as e:
        print(cmd)
        # with open(script.as_posix(), "r") as f:
        #     print(f.read())
        raise e

    if proc.returncode:
        raise RuntimeError(proc)

    return proc.stdout, proc.stderr


def _get_terminal_size() -> Tuple[int, int]:
    # https://stackoverflow.com/a/14422538
    # https://stackoverflow.com/a/18243550
    cols, rows = shutil.get_terminal_size((80, 20))
    return cols, rows


def sudo_pass() -> bytes:
    # Todo:
    #  - [ ] Mechanism to verify that sudo password is correct
    #  - [ ] implement asterisks
    print(" ENTER PASSWORD ".center(_get_terminal_size()[0], "="))
    _sudo_pass = getpass(prompt=f"Sudo Password for User `{getuser()}`: ")
    return _sudo_pass.encode()

## Step 3 - Disable Unattended Upgrades Unit

Disable Unattended Upgrades `systemd` unit.
`sudo`: yes

### Source Write Script Function

In [4]:
import pathlib
import tempfile


def script_disable_unattended_upgrades() -> pathlib.Path:
    print(" DISABLE UNATTENDED UPGRADES ".center(_get_terminal_size()[0], "#"))
    with tempfile.NamedTemporaryFile(
            delete=False,
            encoding="utf-8",
            prefix="ubuntu_2204_",
            suffix=".sh",
            mode="x",
    ) as script:
        script.writelines(
            [
                "#!/bin/env bash\n",
                "\n",
                "\n",
                "sudo systemctl disable --now unattended-upgrades\n",
            ]
        )

        script.writelines(
            [
                "\n",
                "exit 0\n",
            ]
        )

        return pathlib.Path(script.name)

### Run Script

In [5]:
ret_script_disable_unattended_upgrades = script_run(
    sudo=True,
    script=script_disable_unattended_upgrades(),
)

######################### DISABLE UNATTENDED UPGRADES ##########################
----------------------------------- COMMAND ------------------------------------
 /usr/bin/sudo --stdin --reset-timestamp /usr/bin/bash /tmp/ubuntu_2204_b5c_assy.sh 
--------------------------------- SCRIPT START ---------------------------------
1: #!/bin/env bash
2: 
3: 
4: sudo systemctl disable --now unattended-upgrades
5: 
6: exit 0
---------------------------------- SCRIPT END ----------------------------------
================================ ENTER PASSWORD ================================


Sudo Password for User `michael`:  ········


[sudo] password for michael: Failed to disable unit: Unit unattended-upgrades.service does not exist


## Step 4 - Prepare System

`sudo`: yes

### Source Write Script Function

In [7]:
import pathlib
import tempfile


def script_prep() -> pathlib.Path:
    print(" PREP ".center(_get_terminal_size()[0], "#"))
    with tempfile.NamedTemporaryFile(
            delete=False,
            encoding="utf-8",
            prefix=SHELL_SCRIPTS_PREFIX,
            suffix=".sh",
            mode="x",
    ) as script:

        prep_pkgs = [
            "openssh-server",
            "git",
            "htop",
            "vim",
            "graphviz",
        ]
        script.writelines(
            [
                "#!/bin/env bash\n",
                "\n",
                "\n",
                "sudo apt-get update\n",
                "sudo apt-get upgrade -y\n",
                "\n",
                f"sudo apt-get install --no-install-recommends -y {' '.join(prep_pkgs)}\n",
                "\n",
                "sudo apt-get -y autoremove\n",
                "sudo apt-get clean\n",
                "\n",
                "sudo systemctl enable --now ssh\n",
            ]
        )

        script.writelines(
            [
                "\n",
                "exit 0\n",
            ]
        )

        return pathlib.Path(script.name)

### Run Script

In [8]:
ret_script_prep = script_run(
    sudo=True,
    script=script_prep(),
)

##################################### PREP #####################################
----------------------------------- COMMAND ------------------------------------
 /usr/bin/sudo --stdin --reset-timestamp /usr/bin/bash /tmp/ubuntu_desktop_2204__11unx7kh.sh 
--------------------------------- SCRIPT START ---------------------------------
1 : #!/bin/env bash
2 : 
3 : 
4 : sudo apt-get update
5 : sudo apt-get upgrade -y
6 : 
7 : sudo apt-get install --no-install-recommends -y openssh-server git htop vim graphviz
8 : 
9 : sudo apt-get -y autoremove
10: sudo apt-get clean
11: 
12: sudo systemctl enable --now ssh
13: 
14: exit 0
---------------------------------- SCRIPT END ----------------------------------
================================ ENTER PASSWORD ================================


Sudo Password for User `michael`:  ········


[sudo] password for michael: Sorry, try again.
[sudo] password for michael: 
sudo: no password was provided
sudo: 1 incorrect password attempt


['/usr/bin/sudo', '--stdin', '--reset-timestamp', '/usr/bin/bash', '/tmp/ubuntu_desktop_2204__11unx7kh.sh']


CalledProcessError: Command '['/usr/bin/sudo', '--stdin', '--reset-timestamp', '/usr/bin/bash', '/tmp/ubuntu_desktop_2204__11unx7kh.sh']' returned non-zero exit status 1.